In [ ]:
export OS_PROJECT_NAME="$OS_PROJECT_NAME"
export OS_REGION_NAME="CHI@UC"
export NODE_TYPE=compute_skylake
IMAGE=CC-CentOS7

RESOURCE_NAME="rani-cortx-trovi"

In [ ]:
lease_name="$RESOURCE_NAME"
network_name="$RESOURCE_NAME"
public_network_id=$(openstack network show public -f value -c id)

In [ ]:
mkdir -p ~/.ssh
echo "Host *" > ~/.ssh/config
echo " StrictHostKeyChecking no" >> ~/.ssh/config
sudo rm -rf output/*

In [ ]:
blazar lease-create \
  --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation resource_type=virtual:floatingip,network_id="$public_network_id",amount=1 \
  --start-date "$(date +'%Y-%m-%d %H:%M')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$lease_name"

In [ ]:
# Wait for lease to start
timeout 300 bash -c 'until [[ $(blazar lease-show $0 -f value -c status) == "ACTIVE" ]]; do sleep 1; done' "$lease_name" \
    && echo "Lease started successfully!"

In [ ]:
#
# Fetch information about which resources were reserved for later use
#

reservations=$(blazar lease-show "$lease_name" -f json \
  | jq -r '.reservations')
host_reservation_id=$(jq -rs 'map(select(.resource_type=="physical:host"))[].id' <<<"$reservations")
fip_reservation_id=$(jq -rs 'map(select(.resource_type=="virtual:floatingip"))[].id' <<<"$reservations")

fip=$(openstack floating ip list --tags "reservation:$fip_reservation_id" -f json)
fip_id=$(jq -r 'map(.ID)[0]' <<<"$fip")
fip_addr=$(jq -r 'map(.["Floating IP Address"])[0]' <<<"$fip")

In [ ]:
key_pair_upload

In [ ]:
# Create the server instance
openstack server create --wait \
 --flavor baremetal \
 --image "$IMAGE" \
 --hint reservation="$host_reservation_id" \
 --nic net-id=$(openstack network show sharednet1 -f value -c id) \
 --key-name "$USER-jupyter" \
 "$RESOURCE_NAME"

In [ ]:
 # Assign a public IP
openstack server add floating ip "$RESOURCE_NAME" "$fip_addr"

In [ ]:
# In case "No FloatingIP found for null", manually associate the floating ip in Chameleon Web
fip_addr="192.5.87.179"
#openstack floating ip delete 192.5.86.248

In [ ]:
# Wait for public IP connectivity
wait_ssh "$fip_addr"

In [ ]:
ssh cc@"$fip_addr" \
    "realpath . && ls"

In [ ]:
# This will do the job but only 1 commands per "server ..."
# Create a helper function to run commands remotely
server() {
  quoted="$(printf "%q " "$@")"
  ssh cc@"$fip_addr" bash -c "'$quoted'"
}
export -f server

In [ ]:
# MARK --> using server instead of ssh directly [NOT USED]
server cat /etc/os-release # Check if ssh success
server sudo yum update -y
server sudo mkdir -p /mnt/extra
server sudo chown -R cc /mnt/extra
server stat -c "%U %G" /mnt/extra # Check if chown successfull

# Here realising server only for 1 command
server cd /mnt/extra && server realpath . && server git clone --recursive https://github.com/Seagate/cortx-motr.git

In [ ]:
# MARK --> using ssh directly
# 0. Setup zsh [Login on "daniar"]

In [ ]:
# Check if ssh success
ssh cc@"$fip_addr" \
    "cat /etc/os-release && sudo yum update -y"

In [ ]:
# 2. Clone Motr and Install dependencies
ssh cc@"$fip_addr" \
    "  sudo mkdir -p /mnt/extra \
    && sudo chown -R cc /mnt/extra \
    && cd /mnt/extra \
    && git clone --recursive https://github.com/Seagate/cortx-motr.git"

In [ ]:
# install pip and python
sudo yum group install -y "Development Tools"
sudo yum install -y python-devel ansible tmux
curl https://bootstrap.pypa.io/pip/2.7/get-pip.py -o get-pip.py
python get-pip.py pip==19.3.1            
sudo pip install --target=/usr/lib64/python2.7/site-packages ipaddress

# force ansible to use python2
sudo su
echo "all:" >> /etc/ansible/hosts
echo "  ansible_python_interpreter: \"/usr/bin/python2\"" >> /etc/ansible/hosts
exit

# run build dependencies (9 min)
cd /mnt/extra/cortx-motr
sudo ./scripts/install-build-deps

In [ ]:
# 3. Configure libfabric 
    
cd /mnt/extra
wget https://github.com/Seagate/cortx/releases/download/build-dependencies/libfabric-1.11.2-1.el7.x86_64.rpm
wget https://github.com/Seagate/cortx/releases/download/build-dependencies/libfabric-devel-1.11.2-1.el7.x86_64.rpm
sudo rpm -i libfabric-1.11.2-1.el7.x86_64.rpm
sudo rpm -i libfabric-devel-1.11.2-1.el7.x86_64.rpm
sudo sed -i 's|tcp(eth1)|tcp(eth0)|g' /etc/libfab.conf

In [ ]:
# 4. Build motr 
# Make sure that the exampleX.c is in the ORIGINAL form, otherwise, you can't run "make"
# (7 min)
cd /mnt/extra/cortx-motr
sudo ./autogen.sh 

# Disable expensive checks 
    # https://cortxcommunity.slack.com/archives/C019S0SGWNQ/p1607974535256000
    
    sudo ./configure --disable-expensive-checks --disable-immediate-trace  --disable-dev-mode --with-trace-max-level=M0_ERROR
    
    # ./configure --with-trace-max-level=M0_INFO
    # ./configure --help | grep trace

cd /mnt/extra/cortx-motr
sudo make -j48

In [ ]:
# 5. Compile Python util
    
cd /mnt/extra/
sudo yum install -y gcc rpm-build python36 python36-pip python36-devel python36-setuptools openssl-devel libffi-devel python36-dbus
git clone --recursive https://github.com/Seagate/cortx-utils -b main

cd /mnt/extra/cortx-utils
./jenkins/build.sh -v 2.0.0 -b 2
sudo pip3 install -r https://raw.githubusercontent.com/Seagate/cortx-utils/main/py-utils/python_requirements.txt
sudo pip3 install -r https://raw.githubusercontent.com/Seagate/cortx-utils/main/py-utils/python_requirements.ext.txt

cd /mnt/extra/py-utils/dist
sudo yum install -y cortx-py-utils-*.noarch.rpm

In [ ]:
# 6. Building Hare 
    
cd /mnt/extra
# clone repo
git clone https://github.com/Seagate/cortx-hare.git && cd cortx-hare

# install fecter
sudo yum -y install python3 python3-devel yum-utils
sudo yum localinstall -y https://yum.puppetlabs.com/puppet/el/7/x86_64/puppet-agent-7.0.0-1.el7.x86_64.rpm
sudo ln -sf /opt/puppetlabs/bin/facter /usr/bin/facter

# install consul
sudo yum -y install yum-utils
sudo yum-config-manager --add-repo https://rpm.releases.hashicorp.com/RHEL/hashicorp.repo
sudo yum -y install consul-1.9.1

# build and install motr [here
cd /mnt/extra/cortx-motr && time sudo ./scripts/install-motr-service --link
export M0_SRC_DIR=$PWD

# build hare 
cd /mnt/extra/cortx-hare
sudo make
sudo make install

# create hare group
sudo groupadd --force hare
sudo usermod --append --groups hare $USER

# add path to zsh
echo 'PATH=/opt/seagate/cortx/hare/bin:$PATH' >> ~/.zshrc
echo 'export LD_LIBRARY_PATH=/mnt/extra/cortx-motr/motr/.libs/' >> ~/.zshrc
source ~/.zshrc

In [ ]:
# 7. Create loop devices [[AFTER EACH REBOOT]]
    # https://www.thegeekdiary.com/how-to-create-virtual-block-device-loop-device-filesystem-in-linux/
    # If there is only 1 physical storage, you must create loop devices!
        # linux support block device called the loop device, which maps a normal file onto a virtual block device

    # Create a file (25 GB each)

        mkdir -p /mnt/extra/loop-files/
        cd /mnt/extra/loop-files/
        dd if=/dev/zero of=loopbackfile1.img bs=100M count=250
        dd if=/dev/zero of=loopbackfile2.img bs=100M count=250
        dd if=/dev/zero of=loopbackfile3.img bs=100M count=250
        dd if=/dev/zero of=loopbackfile4.img bs=100M count=250
        dd if=/dev/zero of=loopbackfile5.img bs=100M count=250
        
        # check size 
        # du -sh loopbackfile1.img 

            # 1048576000 bytes (1.0 GB) copied, 0.723784 s, 1.4 GB/s
            # 1001M loopbackfile1.img

    # Create the loop device
        cd /mnt/extra/loop-files/
        sudo losetup -fP loopbackfile1.img
        sudo losetup -fP loopbackfile2.img
        sudo losetup -fP loopbackfile3.img
        sudo losetup -fP loopbackfile4.img
        sudo losetup -fP loopbackfile5.img

    # print loop devices 
        losetup -a
            # /dev/loop0: []: (/mnt/extra/loop-files/loopbackfile1.img)
            # /dev/loop1: []: (/mnt/extra/loop-files/loopbackfile2.img)
            # /dev/loop2: []: (/mnt/extra/loop-files/loopbackfile3.img)

    # Format devices into filesystems 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile1.img 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile2.img 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile3.img 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile4.img 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile5.img 

    # mount loop devices

        mkdir -p /mnt/extra/loop-devs/loop0
        mkdir -p /mnt/extra/loop-devs/loop1
        mkdir -p /mnt/extra/loop-devs/loop2
        mkdir -p /mnt/extra/loop-devs/loop3
        mkdir -p /mnt/extra/loop-devs/loop4
        cd /mnt/extra/loop-devs/
        sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop0
        sudo mount -o loop /dev/loop1 /mnt/extra/loop-devs/loop1
        sudo mount -o loop /dev/loop2 /mnt/extra/loop-devs/loop2
        sudo mount -o loop /dev/loop3 /mnt/extra/loop-devs/loop3
        sudo mount -o loop /dev/loop4 /mnt/extra/loop-devs/loop4
        lsblk -f
        df -h 

In [ ]:
lsblk

In [ ]:
# 8. ACTIVATE the lnet [AFTER EACH REBOOT]
    
    # "FIND interface for LNET"
    sudo ifconfig | grep MULTICAST | grep RUNNING
    # use the ethXX address to initiate lnetctl

    # configure Luster (use eth0 which is UP)
    sudo sed -i 's|tcp(eth1)|tcp(eth0)|g' /etc/modprobe.d/lnet.conf
    # sudo sed -i 's|tcp(eth0)|tcp(eth1)|g' /etc/modprobe.d/lnet.conf
    cat /etc/modprobe.d/lnet.conf
    sudo modprobe lnet

    # sudo systemctl stop lnet
    # sudo systemctl start lnet
    # sudo lnetctl net add --net tcp --if eth0

    sudo lctl list_nids

        # sample output: 10.140.81.147@tcp

In [ ]:
# 9. Start a Hare cluster

    # Prepare cdf file 
        cd /mnt/extra
        cp /opt/seagate/cortx/hare/share/cfgen/examples/singlenode.yaml CDF.yaml

        sudo sed -i "s|hostname: localhost|hostname: `hostname`|g" CDF.yaml
        sudo sed -i "s|node: localhost|node: `hostname`|g" CDF.yaml
        sudo sed -i 's|data_iface: eth1|data_iface: eth0|g' CDF.yaml
        # sudo sed -i 's|data_iface: eth0|data_iface: eth1|g' CDF.yaml

        # remove the unavailable storage devices 
        sed -i '/loop0/d' CDF.yaml
        sed -i '/loop1/d' CDF.yaml
        sed -i '/loop2/d' CDF.yaml
        sed -i '/loop3/d' CDF.yaml
        sed -i '/loop4/d' CDF.yaml
        sed -i '/loop8/d' CDF.yaml

        # set the disk for logging
        sudo sed -i "s|loop9|loop8|g" CDF.yaml

        # Check the modification
            cd /mnt/extra
            cat CDF.yaml| grep eth
            cat CDF.yaml| grep hostname
            cat CDF.yaml| grep node:            # only 3 loop devices 

    # bootstrap (0.5 min)
        cd /mnt/extra
        sudo hctl bootstrap --mkfs CDF.yaml
        hctl status
        
        # cd /mnt/extra
        # sudo hctl shutdown; sudo hctl bootstrap --mkfs CDF.yaml

    # check status
        hctl status

        # sample output:
            Data pool:
                # fid name
                0x6f00000000000001:0x0 'the pool'
            Profile:
                # fid name: pool(s)
                0x7000000000000001:0x0 'default': 'the pool' None None
            Services:
                dan-cortx.novalocal  (RC)
                [started]  hax                 0x7200000000000001:0x0          inet:tcp:10.140.81.147@22001
                [started]  confd               0x7200000000000001:0x1          inet:tcp:10.140.81.147@21002
                [started]  ioservice           0x7200000000000001:0x2          inet:tcp:10.140.81.147@21003
                [unknown]  m0_client_other     0x7200000000000001:0x3          inet:tcp:10.140.81.147@22501
                [unknown]  m0_client_other     0x7200000000000001:0x4          inet:tcp:10.140.81.147@22502

In [ ]:
# 10. Connect VSCode ssh 
    # Use the "remote explorer" package to establish remote editor
    # Open the /mnt/extra 
    # Need this to inspect the CDF file, make sure it looks good
    # Also for modifying the motr source code

In [ ]:
# Save variables

cat <<EOF
LEASE_NAME="$LEASE_NAME"
KEYPAIR_NAME="$KEYPAIR_NAME"
SERVER_NAME="$SERVER_NAME"
FLOATING_IP="$FLOATING_IP"
EOF

In [ ]:
exit

In [ ]:
openstack floating ip delete 192.5.86.238

In [ ]:
openstack server delete $RESOURCE_NAME

In [ ]:
blazar lease-delete $RESOURCE_NAME